# Data reading and cleaning

In [1]:
# import libraries and read csv
import pandas as pd
import numpy as np
import re
data = pd.read_csv(r"C:\Users\Vanessa\Downloads\Projeto Final\2016 - 2023.csv", usecols = [0, 1, 2, 4, 5, 7], index_col = None)

In [2]:
# split csv into a list a dataframes
data_list = np.split(data, data[(data.isna().all(axis = 1))|(data.iloc[:,0].str.contains('MÊS'))].index)

In [3]:
# identify not null dataframes from the list, rename columns and add them to another list
clean_data = []
for i in data_list:
    if i.isna().values.all():
        pass
    else:
        if i.iloc[:,0].str.contains('MÊS').any():
            i.columns = ['dia', 'receita', 'depositante', 'despesas', 'favorecido', 'saldo']
            clean_data.append(i)            

In [4]:
# add "month" and "year" columns, reorder columns, drop unwanted rows and save dataframes as excel files
for i in range(len(clean_data)):
    clean_data[i]['mês'] = re.findall('\w+', clean_data[i].iloc[0,1])[0]
    clean_data[i]['ano'] = '20' + (re.findall('\w+', clean_data[i].iloc[0,1])[1]) 
    clean_data[i] = clean_data[i][['ano', 'mês', 'dia', 'receita', 'depositante', 'despesas', 'favorecido', 'saldo']]
    clean_data[i].drop(clean_data[i].index[0:3], inplace = True)
    clean_data[i].to_excel(f'tabela_{i}.xlsx', index = False)

C:\Users\Vanessa\AppData\Local\Temp\ipykernel_24084\450259892.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_table[i].drop(clean_table[i].index[0:3], inplace = True)
C:\Users\Vanessa\AppData\Local\Temp\ipykernel_24084\450259892.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_table[i].drop(clean_table[i].index[0:3], inplace = True)
C:\Users\Vanessa\AppData\Local\Temp\ipykernel_24084\450259892.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

C:\Users\Vanessa\AppData\Local\Temp\ipykernel_24084\450259892.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_table[i].drop(clean_table[i].index[0:3], inplace = True)
C:\Users\Vanessa\AppData\Local\Temp\ipykernel_24084\450259892.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_table[i].drop(clean_table[i].index[0:3], inplace = True)
C:\Users\Vanessa\AppData\Local\Temp\ipykernel_24084\450259892.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

C:\Users\Vanessa\AppData\Local\Temp\ipykernel_24084\450259892.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_table[i].drop(clean_table[i].index[0:3], inplace = True)
C:\Users\Vanessa\AppData\Local\Temp\ipykernel_24084\450259892.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_table[i].drop(clean_table[i].index[0:3], inplace = True)
C:\Users\Vanessa\AppData\Local\Temp\ipykernel_24084\450259892.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

C:\Users\Vanessa\AppData\Local\Temp\ipykernel_24084\450259892.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_table[i].drop(clean_table[i].index[0:3], inplace = True)
C:\Users\Vanessa\AppData\Local\Temp\ipykernel_24084\450259892.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_table[i].drop(clean_table[i].index[0:3], inplace = True)
C:\Users\Vanessa\AppData\Local\Temp\ipykernel_24084\450259892.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

In [4]:
data = pd.read_excel(r"C:\Users\Vanessa\Downloads\Projeto Final\tabela_0.xls")
data

,ano,mês,dia,receita,depositante,despesas,favorecido,saldo
0,2016,Sep,1.0,565.00,Doações,"2,000.00",Salário - Rosa Elisa - AGO/16,"4,480.23"
1,2016,Sep,2.0,350.00,Doações,363.00,Ração - Agropecuaria 101,"4,467.23"
2,2016,Sep,2.0,NaN,NaN,10.00,Combust.Posto S. Cristovão,"4,457.23"
3,2016,Sep,2.0,NaN,NaN,143.57,Combust.Posto Shark,"4,313.66"
4,2016,Sep,2.0,NaN,NaN,50.00,Saque no TAA,"4,263.66"
...,...,...,...,...,...,...,...,...
56,2016,Sep,9.0,NaN,NaN,50.00,Combust. P. Vitória,"4,188.26"
57,2016,Sep,16.0,NaN,NaN,50.00,Combust.Auto PostoN.Camp.,"4,138.26"
58,2016,Sep,20.0,NaN,NaN,"2,901.60",GPS - INSS,"1,236.66"
59,2016,Sep,20.0,NaN,NaN,274.39,DARF - Receita Federal,962.27
